In [1]:
import sqltables.sqlite3
import datetime
from tqdm.notebook import tqdm
import openreview
import json

In [2]:
with open("openreview_credentials.json") as fh:
    credentials = json.load(fh)
    username = credentials["username"]
    password = credentials["password"]

client = openreview.api.OpenReviewClient(
    baseurl='https://api2.openreview.net',
    username=username,
    password=password
)

In [3]:
venues = client.get_group(id='venues').members
print(list(reversed(sorted([v for v in venues if "ICML" in v and "Conference" in v]))))


['ICML.cc/2025/Conference', 'ICML.cc/2024/Conference', 'ICML.cc/2023/Conference', 'ICML.cc/2020/Conference']


In [4]:
venue_id = "ICML.cc/2025/Conference"

In [5]:
db = sqltables.sqlite3.Database("or_submissions.sqlite3")
if "submissions" in list(db.tables):
    submissions = db.open_table("submissions")
else:
    submissions = db.create_table(name="submissions", column_names=["arxiv_id", "date", "title", "authors", "url", "abstract", "categories"])

In [6]:
submissions.view("select max(arxiv_id) from _")

|max\(arxiv\_id\)|
|-|
|\'zxg6601zoc\'|


In [7]:
processed_submissions = set(row.arxiv_id for row in submissions)
len(processed_submissions)

3704

In [8]:
# Get all
# venue_group = client.get_group(venue_id)
# submission_name = venue_group.content['submission_name']['value']
# print(submission_name)
# submissions = client.get_all_notes(invitation=f'{venue_id}/-/{submission_name}')

# Get accepted submissions
or_submissions = client.get_all_notes(content={'venueid': venue_id} )
len(or_submissions)

0

In [73]:
rows = []
for s in or_submissions:
    arxiv_id = s.id    
    if arxiv_id in processed_submissions:
        continue    
    dt = datetime.datetime.fromtimestamp(s.pdate / 1000)
    date = dt.isoformat()
    content = s.content    
    title = content["title"]["value"]
    url = f"https://openreview.net/forum?id={arxiv_id}"
    abstract = content["abstract"]["value"]
    authors = ", ".join(content["authors"]["value"])
    categories = venue_id
    rows.append([arxiv_id, date, title, authors, url, abstract, categories])
len(rows)

3704

In [74]:
rows[0]

['zxg6601zoc',
 '2025-01-22T17:10:26.953000',
 'Re-Imagining Multimodal Instruction Tuning: A Representation View',
 'Yiyang Liu, James Chenhao Liang, Ruixiang Tang, Yugyung Lee, MAJID RABBANI, Sohail Dianat, Raghuveer Rao, Lifu Huang, Dongfang Liu, Qifan Wang, Cheng Han',
 'https://openreview.net/forum?id=zxg6601zoc',
 'Multimodal instruction tuning has proven to be an effective strategy for achieving zero-shot generalization by fine-tuning pre-trained Large Multimodal Models (LMMs) with instruction-following data. However, as the scale of LMMs continues to grow, fully fine-tuning these models has become highly parameter-intensive. Although Parameter-Efficient Fine-Tuning (PEFT) methods have been introduced to reduce the number of tunable parameters, a significant performance gap remains compared to full fine-tuning. Furthermore, existing PEFT approaches are often highly parameterized, making them difficult to interpret and control. In light of this, we introduce Multimodal Representa

In [75]:
submissions.insert(rows)

In [76]:
submissions.view("select count(*) from _")

|count\(\*\)|
|-|
|3704|


In [77]:
submissions.view("select count(distinct arxiv_id) from _")

|count\(distinct arxiv\_id\)|
|-|
|3704|


In [78]:
db.close()